In [1]:
import math
import pandas as pd
import numpy as np

class FedBatchBioreactor(object):
    
    def __init__(self,MassProduced,Titre,Disposable):
        self.MassProduced = MassProduced
        self.Titre = Titre
        self.Disposable = Disposable
        
    def Sizing(self):
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        #Production bioreactor working volume
        PBioreactorWV=self.MassProduced*1000/self.Titre
        #Cell culture media volume
        CCMediaVolume=PBioreactorWV*(1-Protocol.get_value('Inoculation Ratio','Bioreactor')+Protocol.get_value('Fed-batch Media addition','Bioreactor'))*(1+Protocol.get_value('Media Overfill','Hold-Tanks'))
        return PBioreactorWV,CCMediaVolume
    
    def MassBalance(self):
        #Mass out
        MassOut=self.MassProduced
        #Volume Out
        VolumeOut=FedBatchBioreactor.Sizing(self)[0]
        #Concentration Out
        ConcOut=self.Titre
        return MassOut,VolumeOut,ConcOut
    
    def Timeframe(self):
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        #time to prepare for a batch
        PrepTime=Protocol.get_value('Preparation Time (hr)','Bioreactor')+Protocol.get_value('Load Time (hr)','Bioreactor')
        #time to operate a batch
        OperTime=Protocol.get_value('Operation Time (hr)','Bioreactor')
        #time to clean
        CIPTime=Protocol.get_value('CIP Time (hr)','Bioreactor')+Protocol.get_value('SIP Time (hr)','Bioreactor')
        #time to turnaround bioreactors for next batch
        TurnAroundTime=Protocol.get_value('Turnaround Time (hr)','Bioreactor')
        #time to turnaround hold-tanks
        HoldTanksCIPandSIPTime=Protocol.get_value('CIP Time (hr)','Hold-Tanks')+Protocol.get_value('SIP Time (hr)','Hold-Tanks')
        return PrepTime,OperTime,CIPTime,TurnAroundTime,HoldTanksCIPandSIPTime
    
    def EquipmentAndMaterials(self):
        #Load Equipment from Database
        Equipment=pd.read_excel('Database.xlsx',sheetname='Equipment',index_col=0)
        #Load Materials from Database
        Materials=pd.read_excel('Database.xlsx',sheetname='Materials',index_col=0)
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        #CC Media hold-tank with GF, PW, WFI, CIP
        CCMediaCost=FedBatchBioreactor.Sizing(self)[1]*Materials.get_value('Cell Culture Media','Base Cost')
        if FedBatchBioreactor.Sizing(self)[1]/Protocol.get_value('Space Efficiency','Hold-Tanks')>Equipment.get_value('Hold-Tank','Max. Size'):
            NumberOfMediaTanks=math.ceil((FedBatchBioreactor.Sizing(self)[1]/Protocol.get_value('Space Efficiency','Hold-Tanks'))/Equipment.get_value('Hold-Tank','Max. Size'))
        else:
            NumberOfMediaTanks=1
        MediaTankVolume=(FedBatchBioreactor.Sizing(self)[1]/Protocol.get_value('Space Efficiency','Hold-Tanks'))/NumberOfMediaTanks
        if MediaTankVolume<=10:
            CorrectedMediaTankVolume=math.ceil(MediaTankVolume)
        elif MediaTankVolume<=100 and MediaTankVolume>10:
            CorrectedMediaTankVolume=int(math.ceil(MediaTankVolume/10.0))*10
        elif MediaTankVolume<=1000 and MediaTankVolume>100:
            CorrectedMediaTankVolume=int(math.ceil(MediaTankVolume/100.0))*100
        else:
            CorrectedMediaTankVolume=int(math.ceil(MediaTankVolume/1000.0))*1000
        MediaHoldTankCost=NumberOfMediaTanks*Equipment.get_value('Hold-Tank','Base Cost')*(CorrectedMediaTankVolume/Equipment.get_value('Hold-Tank','Base Size'))**Equipment.get_value('Hold-Tank','Scaling Factor')
        MediaHoldTankGuard=NumberOfMediaTanks*Materials.get_value('Hold-Tank Guard Filter','Base Cost')*(CorrectedMediaTankVolume/Materials.get_value('Hold-Tank Guard Filter','Base Size'))**Materials.get_value('Hold-Tank Guard Filter','Scaling Factor')
        MediaCIPBuffer=NumberOfMediaTanks*(Protocol.get_value('Acid Buffer','Hold-Tanks')+Protocol.get_value('Caustic Buffer','Hold-Tanks'))*CorrectedMediaTankVolume*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
        MediaCIPBufferCost=MediaCIPBuffer*Materials.get_value('CIP Buffer','Base Cost')
        MediaHoldTankDiameter=NumberOfMediaTanks*(CorrectedMediaTankVolume*4000/(math.pi*Protocol.get_value('H/D tank ratio','Hold-Tanks')))**(1/3)
        MediaWFIVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*MediaHoldTankDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Hold-Tanks')/60*Protocol.get_value('WFI Rinse (min)','Hold-Tanks')
        MediaPWVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*MediaHoldTankDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Hold-Tanks')/60*Protocol.get_value('PW Rinse (min)','Hold-Tanks')
        MediaWFIandPWCost=MediaWFIVolume*Materials.get_value('WFI','Base Cost')+MediaPWVolume*Materials.get_value('PW','Base Cost')
        #Bioreactor with PW, WFI, CIP
        if FedBatchBioreactor.Sizing(self)[0]/Protocol.get_value('Space Efficiency','Bioreactor')>Equipment.get_value('Bioreactor','Max. Size'):
            NumberOfBioreactors=math.ceil((FedBatchBioreactor.Sizing(self)[0]/Protocol.get_value('Space Efficiency','Bioreactor'))/Equipment.get_value('Bioreactor','Max. Size'))
        else:
            NumberOfBioreactors=1
        BioreactorVolume=(FedBatchBioreactor.Sizing(self)[0]/Protocol.get_value('Space Efficiency','Bioreactor'))/NumberOfBioreactors
        if BioreactorVolume<=10:
            CorrectedBioreactorVolume=math.ceil(BioreactorVolume)
        elif BioreactorVolume<=100 and BioreactorVolume>10:
            CorrectedBioreactorVolume=int(math.ceil(BioreactorVolume/10.0))*10
        elif BioreactorVolume<=1000 and BioreactorVolume>100:
            CorrectedBioreactorVolume=int(math.ceil(BioreactorVolume/100.0))*100
        else:
            CorrectedBioreactorVolume=int(math.ceil(BioreactorVolume/1000.0))*1000
        BioreactorCost=NumberOfBioreactors*Equipment.get_value('Bioreactor','Base Cost')*(CorrectedBioreactorVolume/Equipment.get_value('Bioreactor','Base Size'))**Equipment.get_value('Bioreactor','Scaling Factor')
        BioreactorCIPBuffer=NumberOfBioreactors*(Protocol.get_value('Acid Buffer','Bioreactor')+Protocol.get_value('Caustic Buffer','Bioreactor'))*CorrectedBioreactorVolume*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
        BioreactorCIPBufferCost=BioreactorCIPBuffer*Materials.get_value('CIP Buffer','Base Cost')
        BioreactorDiameter=NumberOfBioreactors*(CorrectedBioreactorVolume*4000/(math.pi*Protocol.get_value('H/D tank ratio','Bioreactor')))**(1/3)
        BioreactorWFIVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*BioreactorDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Bioreactor')/60*Protocol.get_value('WFI Rinse (min)','Bioreactor')
        BioreactorPWVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*BioreactorDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Bioreactor')/60*Protocol.get_value('PW Rinse (min)','Bioreactor')
        BioreactorWFIandPWCost=BioreactorWFIVolume*Materials.get_value('WFI','Base Cost')+BioreactorPWVolume*Materials.get_value('PW','Base Cost')
        return BioreactorCost,MediaHoldTankCost,CCMediaCost,MediaCIPBufferCost,MediaWFIandPWCost,BioreactorCIPBufferCost,BioreactorWFIandPWCost,MediaHoldTankGuard,NumberOfBioreactors,CorrectedBioreactorVolume
    
    def CostBreakdown(self):
        EquipmentPurchaseCost=sum(FedBatchBioreactor.EquipmentAndMaterials(self)[0:2])
        #costs per batch
        ChemicalReagentsCost=sum(FedBatchBioreactor.EquipmentAndMaterials(self)[2:7])
        ConsumablesCost=sum(FedBatchBioreactor.EquipmentAndMaterials(self)[7:8])
        LabourCost=0
        return LabourCost,ChemicalReagentsCost,ConsumablesCost,EquipmentPurchaseCost


In [2]:
CellCulture=FedBatchBioreactor(Disposable=False,MassProduced=10,Titre=3)
print(CellCulture.Sizing())
print(CellCulture.MassBalance())
print(CellCulture.Timeframe())
print(CellCulture.EquipmentAndMaterials())
print(CellCulture.CostBreakdown())

(3333.3333333333335, 4408.333333333333)
(10, 3333.3333333333335, 3)
(8.0, 336.0, 7.0, 48.0, 6.0)
(826076.38091843866, 103864.13051294378, 220416.66666666666, 20700.0, 466.26886037188666, 17250.0, 383.30596444448491, 885.05567918349891, 1, 5000)
(0, 259216.24149148303, 885.05567918349891, 929940.5114313825)
